# Compiling Go Source Code

In [1]:
!cat test.go

package main

import (
	"github.com/uuosio/chain"
)

//contract hello
type MyContract struct {
	Receiver      chain.Name
	FirstReceiver chain.Name
	Action        chain.Name
}

func NewContract(receiver, firstReceiver, action chain.Name) *MyContract {
	return &MyContract{receiver, firstReceiver, action}
}

//action sayhello
func (c *MyContract) SayHello(name string) {
	chain.Println("Hello", name)
}


In [2]:
!eosio-go build -o test.wasm .

parse file: test.go
++struct: MyContract
+++action sayhello
wasm-ld --allow-undefined --stack-first --export-dynamic --no-demangle -zstack-size=8192 -L /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gscdk/tinygo -o /var/folders/pv/678xdw3s56xggsvz36r65kzr0000gp/T/tinygo3264904622/main /var/folders/pv/678xdw3s56xggsvz36r65kzr0000gp/T/tinygo3264904622/main.o /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gscdk/tinygo/lib/wasi-libc-eosio/sysroot/lib/wasm32-wasi/libc.a


In [3]:
!ls -l *.wasm

-rwxr-xr-x  1 newworld  staff  1835 Oct  9 23:47 test.wasm


In [4]:
!ls -l *.abi

-rw-r--r--  1 newworld  staff  563 Oct  9 23:47 hello.abi


# Testing

In [5]:
import os
from pyeoskit import eosapi, wallet, config

python_contract = config.python_contract
test_account1 = 'helloworld11'
test_account2 = 'helloworld12'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')


# active key of helloworld11
wallet.import_key('test', '5JRYimgLBrRLCBAcjHUWCYRv3asNedTYYzVgmiU4q2ZVxMBiJXL')
wallet.import_key('test', '5Jbb4wuwz8MAzTB9FJNmrVYGXo4ABb7wqPVoWGcZ6x8V2FwNeDo')
# active key of helloworld12
wallet.import_key('test', '5JHRxntHapUryUetZgWdd3cg6BrpZLMJdqhhXnMaZiiT4qdJPhv')

config.python_contract = 'hello'
config.main_token = 'EOS'
config.main_token_contract = 'eosio.token'
eosapi.set_node('https://testnode.uuos.network:8443')

In [6]:
with open('test.go', 'r') as f:
    code = f.read()
code, abi = eosapi.compile('hello', code, src_type=2)
print('done!')

done!


In [7]:
try:
    eosapi.deploy_contract(test_account1, code, abi, vm_type=0)
except Exception as e:
    print(e.json['error']['what'])
print('done!')

done!


In [8]:
r = eosapi.push_action(test_account1, 'sayhello', {'name': 'alice'}, {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])

Hello alice

645
